This Notebook containts different model and ensemble technique that we have tried.

In [21]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn import svm
import xgboost as xgb
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import confusion_matrix, roc_auc_score ,roc_curve,auc
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import NearMiss
from collections import Counter
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
import re
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import AdaBoostClassifier
from catboost import CatBoostRegressor, CatBoostClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.ensemble import  VotingClassifier

plt.style.use('seaborn')
pd.set_option('display.max_column',100)
pd.options.display.max_rows = 100

In [23]:
TrainData = pd.read_csv('../input/airbnb-new-user/train.csv')
TestData = pd.read_csv('../input/airbnb-new-user/test.csv')
SessData = pd.read_csv('../input/airbnb-new-user/sessions.csv')
SessData.rename(columns={'user_id':'id'},inplace = True)

In [24]:
action_type = pd.pivot_table(SessData, index = ['id'],columns = ['action_type'],values = 'secs_elapsed',aggfunc=len,fill_value=0).reset_index()
action = pd.pivot_table(SessData, index = ['id'],columns = ['action'],values = 'secs_elapsed',aggfunc=len,fill_value=0).reset_index()
action_detail = pd.pivot_table(SessData, index = ['id'],columns = ['action_detail'],values = 'secs_elapsed',aggfunc=len,fill_value=0).reset_index()
device_type = pd.pivot_table(SessData, index = ['id'],columns = ['device_type'],values = 'secs_elapsed',aggfunc=len,fill_value=0).reset_index()

In [25]:
DataCom = pd.concat((TrainData.drop('country_destination',axis=1),TestData),axis=0,ignore_index=True)
def set_age_group(x):
    if x < 40:
        return 'Young'
    elif x >=40 and x < 60:
        return 'Middle'
    elif x >= 60 and x <= 125:
        return 'Old'
    else:
        return 'Unknown'


def set_age(x):
    if x>=16 and x<=120:
        return x
    elif x<16:
        return np.nan
    elif x>120:
        if 2015-x>16 and 2015-x<110:
            return 2015-x
        else:
            return np.nan

def set_categorial_values(x):
    l = ['first_browser','affiliate_provider','first_device_type','affiliate_channel','first_affiliate_tracked','signup_app']
    thresold = [0.00]*10
    
    i = l.index(x)
    l1 = DataCom[x].value_counts(normalize=True)
    l2 = l1[l1>thresold[i]].index.tolist()
    return DataCom[x].apply(lambda x: x if x in l2 else 'diff')

def feature_engineering(df):
    df['DAC_year'] = np.vstack(df['date_account_created'].astype(str).apply(lambda x: list(map(int, x.split('-')))).values)[:,0]
    df['DAC_month'] = np.vstack(df['date_account_created'].astype(str).apply(lambda x: list(map(int, x.split('-')))).values)[:,1]
    df['DAC_day'] = np.vstack(df['date_account_created'].astype(str).apply(lambda x: list(map(int, x.split('-')))).values)[:,2]
    df['DAC_dayofweek'] = pd.to_datetime(df['date_account_created']).dt.dayofweek
    
    df['TFA_year'] = np.vstack(df.timestamp_first_active.astype(str).apply(lambda x: list(map(int, [x[:4],x[4:6],x[6:8],x[8:10],x[10:12],x[12:14]]))).values)[:,0]
    df['TFA_month'] = np.vstack(df.timestamp_first_active.astype(str).apply(lambda x: list(map(int, [x[:4],x[4:6],x[6:8],x[8:10],x[10:12],x[12:14]]))).values)[:,1]
    df['TFA_day'] = np.vstack(df.timestamp_first_active.astype(str).apply(lambda x: list(map(int, [x[:4],x[4:6],x[6:8],x[8:10],x[10:12],x[12:14]]))).values)[:,2]
    df['TFA_hour'] = np.vstack(df.timestamp_first_active.astype(str).apply(lambda x: list(map(int, [x[:4],x[4:6],x[6:8],x[8:10],x[10:12],x[12:14]]))).values)[:,3]
    
    df['DFB_year'] = np.vstack(df['date_first_booking'].fillna(-1).astype(str).apply(lambda x: -1 if x=='-1' else list(map(int,x.split('-')))[0] ))
    df['DFB_month'] = np.vstack(df['date_first_booking'].fillna(-1).astype(str).apply(lambda x: -1 if x=='-1' else list(map(int,x.split('-')))[1] ))
    df['DFB_day'] = np.vstack(df['date_first_booking'].fillna(-1).astype(str).apply(lambda x: -1 if x=='-1' else list(map(int,x.split('-')))[2] ))
    df['DFB_dayofweek'] = pd.to_datetime(df['date_first_booking']).dt.dayofweek
    
    df['lag'] = (pd.to_datetime(df.date_account_created)-pd.to_datetime(df.timestamp_first_active,format='%Y%m%d%H%M%S')).dt.days
    
    df['age'] = df['age'].apply(set_age)
    df['age_group'] = df['age'].apply(set_age_group)
    df['age'] = df['age'].fillna(-1)
    
    
    df['has_booked'] = df['date_first_booking'].fillna(-1).apply(lambda x: 0 if x==-1 else 1)
    df['first_affiliate_tracked'] = df['first_affiliate_tracked'].fillna('unknown')
    
    l = ['first_browser','affiliate_provider','first_device_type','affiliate_channel','first_affiliate_tracked','signup_app']
    for x in l:
        df[x] = set_categorial_values(x)
            
    ohe = ['gender', 'signup_method', 'language', 'affiliate_channel', 'affiliate_provider', 
           'first_affiliate_tracked', 'signup_app', 'first_device_type', 'first_browser','age_group']
    
    for x in ohe:
        combined_data,_ = pd.factorize(df[x],sort=True)
        combined_data = pd.Series(combined_data).astype('int32')
        df[x] = combined_data.values  
    
    droplist = ['date_account_created','timestamp_first_active','date_first_booking','signup_method']
    df = df.drop(droplist,axis=1)
    
    df = pd.merge(df, action_type[['id','booking_response','-unknown-']], how='left', on='id')
    df = pd.merge(df, action[['id','requested', 'confirm_email', 'update', 'cancellation_policies']], how='left', on='id')
    df = pd.merge(df, action_detail[['id','pending', 'at_checkpoint']], how='left', on='id')
    
    df = df.set_index('id')
    
    return df


DataCom = feature_engineering(DataCom)

In [26]:
y = TrainData['country_destination']             
labels = y.values
le = LabelEncoder()
y = pd.Series(le.fit_transform(labels))      

X = DataCom[:TrainData.shape[0]]                        # encoded train data x               
df_test = DataCom[TrainData.shape[0]:]                  # encoded test data

x_train, x_test, y_train, y_test = train_test_split(X, y,random_state=31, train_size=0.80,  stratify=y)

In [27]:
xgb3 = xgb.XGBClassifier(max_depth=4, learning_rate=0.01, n_estimators=75,
                    objective='multi:softprob', subsample=0.6, colsample_bytree=0.6, seed=40,reg_lambda=0.5)
xgb3.fit(X, y)
print(xgb3.score(x_test,y_test))

0.8738391912542612


In [28]:
cat1 = CatBoostClassifier(iterations=50,learning_rate = 0.01)
cat1.fit(X,y,eval_set=(x_test, y_test))
print(cat1.score(x_test,y_test))

0:	learn: 2.4002534	test: 2.4002566	best: 2.4002566 (0)	total: 430ms	remaining: 21.1s
1:	learn: 2.3246206	test: 2.3246177	best: 2.3246177 (1)	total: 884ms	remaining: 21.2s
2:	learn: 2.2560967	test: 2.2560883	best: 2.2560883 (2)	total: 1.27s	remaining: 19.9s
3:	learn: 2.1935011	test: 2.1934946	best: 2.1934946 (3)	total: 1.6s	remaining: 18.4s
4:	learn: 2.1361821	test: 2.1361654	best: 2.1361654 (4)	total: 2.04s	remaining: 18.4s
5:	learn: 2.0829798	test: 2.0829698	best: 2.0829698 (5)	total: 2.48s	remaining: 18.2s
6:	learn: 2.0334779	test: 2.0334691	best: 2.0334691 (6)	total: 2.8s	remaining: 17.2s
7:	learn: 1.9872891	test: 1.9872821	best: 1.9872821 (7)	total: 3.25s	remaining: 17.1s
8:	learn: 1.9440081	test: 1.9440038	best: 1.9440038 (8)	total: 3.7s	remaining: 16.8s
9:	learn: 1.9033506	test: 1.9033355	best: 1.9033355 (9)	total: 4.15s	remaining: 16.6s
10:	learn: 1.8648835	test: 1.8648658	best: 1.8648658 (10)	total: 4.53s	remaining: 16.1s
11:	learn: 1.8285131	test: 1.8284930	best: 1.8284930 (1

In [29]:
lgbmodel1 = LGBMClassifier(learning_rate=0.1,n_estimators=300)
lgbmodel1.fit(X,y)
lgbmodel1.score(x_test,y_test)

0.8858587045962149

In [30]:
stack = StackingClassifier(estimators=[('xgb',xgb3),('cat',CatBoostClassifier(iterations=5))], final_estimator=LogisticRegression(),cv=5)
stack.fit(X,y)
stack.score(x_test,y_test)

Learning rate set to 0.5
0:	learn: 0.5756437	total: 322ms	remaining: 1.29s
1:	learn: 0.5074613	total: 764ms	remaining: 1.15s
2:	learn: 0.4980512	total: 1.22s	remaining: 810ms
3:	learn: 0.4926831	total: 1.75s	remaining: 437ms
4:	learn: 0.4890949	total: 2.42s	remaining: 0us
Learning rate set to 0.5
0:	learn: 0.5745474	total: 301ms	remaining: 1.2s
1:	learn: 0.5057818	total: 687ms	remaining: 1.03s
2:	learn: 0.4964825	total: 1.05s	remaining: 704ms
3:	learn: 0.4907251	total: 1.5s	remaining: 376ms
4:	learn: 0.4873036	total: 1.92s	remaining: 0us
Learning rate set to 0.5
0:	learn: 0.5755796	total: 316ms	remaining: 1.26s
1:	learn: 0.5073235	total: 726ms	remaining: 1.09s
2:	learn: 0.4941844	total: 1.18s	remaining: 785ms
3:	learn: 0.4889438	total: 1.6s	remaining: 399ms
4:	learn: 0.4857008	total: 2s	remaining: 0us
Learning rate set to 0.5
0:	learn: 0.5756204	total: 274ms	remaining: 1.1s
1:	learn: 0.5108733	total: 729ms	remaining: 1.09s
2:	learn: 0.4920953	total: 1.14s	remaining: 761ms
3:	learn: 0.4

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


0.8738391912542612

In [31]:
max_vot = VotingClassifier(estimators=[('xgb', xgb3), ('cat', CatBoostClassifier(iterations=5))],voting='soft')
max_vot.fit(X,y)
print(max_vot.score(x_test,y_test))

Learning rate set to 0.5
0:	learn: 0.5756437	total: 419ms	remaining: 1.68s
1:	learn: 0.5074613	total: 1.06s	remaining: 1.59s
2:	learn: 0.4980512	total: 1.61s	remaining: 1.07s
3:	learn: 0.4926831	total: 2.11s	remaining: 527ms
4:	learn: 0.4890949	total: 2.58s	remaining: 0us
0.8738391912542612


In [32]:
prob = xgb3.predict_proba(df_test)
id_test = TestData['id']

id_final_sub = []  
pred_final_sub = [] 
for i in range(len(id_test)):
    idx = id_test[i]
    id_final_sub += [idx] * 3
    pred_final_sub += le.inverse_transform(np.argsort(prob[i])[::-1])[:3].tolist()

sub = pd.DataFrame(np.column_stack((id_final_sub,pred_final_sub)), columns=['id', 'country_destination'])
sub.to_csv('submission.csv',index=False)